In [1]:
import pandas_datareader as pdr
import pandas as pd
import datetime

ticker = "^GSPC"
start=datetime.date(1980, 1, 1)
end=datetime.date(2019, 1, 1)
sp500 = pdr.get_data_yahoo(ticker, start, end)
sp500.to_csv('sp500.csv')

In [3]:
len(sp500)

9835

In [14]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

class data_manager():
	def __init__(self):
		self.csv_file = 'sp500.csv'
		pass

#where to normalize?
	def prepare_pd(self):
		self.read_csv()
		self.parse_date()
		self.data_begin_date = None
		self.data_end_date = None
		self.add_columns()

	def add_columns(self,is_financial = True, contain_adjusted = True,contain_volume=True,validation_split=0.81):
		if is_financial:
			if contain_adjusted:
				self.data['Log Return'] = np.log(self.data['Adj Close']) - np.log(self.data['Adj Close'].shift(1))
				self.data['Pct Change'] = self.data['Adj Close'].pct_change().dropna()
			else:
				self.data['Log Return'] = np.log(self.data['Close']) - np.log(self.data['Close'].shift(1))
				self.data['Pct Change'] = self.data['Close'].pct_change().dropna()

			self.data = self.data.dropna()


	def read_csv(self):
		self.data = pd.read_csv(self.csv_file)

	def get_pd_table(self,**kwargs):
		table = self.data
		if 'date_start' in kwargs:
			table = table.ix[table['Date'] > kwargs['date_start']]
			kwargs.pop('date_start')
		if 'date_end' in kwargs:
			table = table.ix[self.data['Date'] < kwargs['date_end']]
			kwargs.pop('date_end')

		return table

	def parse_date(self):
		self.data['Date'] = pd.to_datetime(self.data['Date'])
		oldtime_table = self.data.copy()
		oldtime_table['Date'] -= pd.Timedelta(100*365,'D')
		self.data = self.data.where(self.data['Date'] < pd.to_datetime('2017/1/1'),oldtime_table)